In [26]:
import os
import time
from urllib import request
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
import pandas as pd

In [27]:
# Function to get price from the search result page
def get_price(soup):
    price = ""
    sign_list = soup.find_all(text = re.compile('₹'))
    pattern = re.compile(r'(?<=₹)\ ?\d+\,?\d+')
    #print(sign_list)
    if len(sign_list)>0:
        for sign in sign_list:
            if len(pattern.findall(sign))>0:
                price = int(pattern.findall(sign)[0].replace(',',""))
                if price>2000: #remove irrelavant products (such as phone cover)
                    break;
    else: 
        rs_list = soup.find_all(text = re.compile('Rs.'))
        pattern = re.compile(r'(?<=Rs.)\ ?\d+\,?\d+')
        if len(rs_list)>0:
            for rs in rs_list:
                if len(pattern.findall(rs))>0:
                    price = int(pattern.findall(rs)[0].replace(',',""))
                    if price>2000:
                        break;    
        else:
            price = ""
    #print(price)
    return price
                

In [28]:
# Function to operate chrome driver to search device keywords
def search_price(device):
    search = driver.find_element_by_name("q")
    search.clear()
    search.send_keys(device + " price india")
    search.send_keys(Keys.RETURN)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "lxml")
    price = get_price(soup)
    return price

In [29]:
# Load list
data = pd.read_csv('sample_data.csv')

In [30]:
data['selenium_download'] = pd.Series()

In [31]:
selenium_download = data['selenium_download'].copy()

In [32]:
data.head()

,device_model_name,selenium_download
0,Redmi Note 5 Pro,NaN
1,Redmi Note 4,NaN
2,Redmi Note 7 Pro,NaN
3,ONEPLUS A6000,NaN
4,Redmi Note 6 Pro,NaN


In [33]:
device = data.device_model_name

In [34]:
def run_search_by_ind(device, selenium_download, ind):
    """Main function to start downloading: 
    loop through the device lists within the index range, and fill the price into the series。
    """
    for i in ind:
        print(device[i])
        result = search_price(device[i])
        print('price:', result)
        selenium_download[i] = result
    return None

In [35]:
# Set up chromedriver
PhantomPath = "chromedriver.exe"  # Chromedriver should be up-to-date. Update at https://chromedriver.chromium.org/.
driver =webdriver.Chrome(executable_path=PhantomPath)
driver.set_page_load_timeout(300)
time.sleep(6)
driver.get("https://www.google.com")
time.sleep(6)

In [ ]:
# Single operation. Use for debugging
# search_price(device[1])

In [36]:
# Set download index range
ind = device[0:20].index

In [37]:
run_search_by_ind(device, selenium_download, ind)

Redmi Note 5 Pro
price: 11790
Redmi Note 4
price: 8499
Redmi Note 7 Pro
price: 13990
ONEPLUS A6000
price: 29999
Redmi Note 6 Pro
price: 11157
ONEPLUS A6010
price: 30999
SM-A505F
price: 15899
SM-G610F
price: 5999
Redmi Note 8 Pro
price: 18890
iPhone8,1
price: 39900
GM1901
price: 29950
iPhone9,3
price: 75990
Redmi 4
price: 4399
Redmi Note 5
price: 5799
iPhone7,2
price: 23990
POCO F1
price: 19999
SM-M307F
price: 16999
Redmi 5A
price: 7900
iPhone11,8
price: 57504
HD1901
price: 39999


In [38]:
data['selenium_download'] = selenium_download

In [39]:
# Save data
data.to_csv('selenium_download.csv')